<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/2_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 테스트

- osmu 테스트
- CIFAR10 + resnet18


##2.1.구글 Colab 기반 

이그나이트는 작성 시점을 기준으로 다음의 백엔드들을 지원한다.

- backends from native torch distributed configuration: “nccl”, “gloo”, “mpi”
- XLA on TPUs via pytorch/xla
- using Horovod framework as a backend

각 의미를 이해하기 위해 가능한 단순한 내용의 코드를 준비하고, 이를 구글 Colab 기반으로 동작시켜 본다.


### 2.1.4. 분산 딥러닝 예제

 지금까지는 sort 반복과 같이 간단한 작업을, colab 기반의 다양한 환경(CPU, GPU, TPU)에서 여러 개의 프로세스로 나누어 분할 실행시켜 보았다. 이제는 실제 딥러닝 모델을 준비하고, 이를 단일코드 형태를 유지한 채로 다양한 환경(CPU, GPU, TPU)에서 학습시키는 방법을 살펴본다. 



####2.1.4.1. Colab runtime 구분

 분산처리를 진행하는 경우, TPU에 한해, colab runtime을 TPU 구동이 가능한 VM으로 환경을 설정해주는 작업이  필요하다. 이를 위해 아래와 같은 코드로 colab의 runtime type이 TPU인지 먼저 확인한다. 그리고 검출된 환경이 TPU인 경우, 분산처리가 가능한 런타임으로 변경하는 작업을 수행한다. 참고로 GPU 사용 시에는 런타임 유형 변경에서 GPU를 선택하는 것만으로 GPU를 사용할 수 있다. 

 다행히 파이토치 커뮤니티에서는 아래와 같이 사용이 편리한 xla 설정 관련 스크립트를 제공하고 있다. 

 ** 가이드라인 작성 시점 기준 잔여 [issue](https://github.com/pytorch/builder/pull/750)로, 20210304 version으로 지정함


In [1]:
import os

gpu_gtg = False
if int(os.environ.get("COLAB_GPU")) > 0:
    gpu_gtg = "COLAB_GPU" in os.environ

tpu_gtg = "COLAB_TPU_ADDR" in os.environ

if tpu_gtg: # tpu
    print("TPU")
    #VERSION = "nightly"

    # https://github.com/pytorch/builder/pull/750
    VERSION = "20210304" # was 20200607" 

    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

TPU
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  34335      0 --:--:-- --:--:-- --:--:-- 34335
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20210304 ...
Found existing installation: torch 1.9.0+cu102
     |████████████████████████████████| 57 kB 1.9 MB/s 
Uninstalling torch-1.9.0+cu102:
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.278 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompa

####2.1.4.2. 패키지 설치

 다음과 같이 이그나이트의 최신 version을 설치한다. 현 시점 기준 Colab에서 제공하는 VM은, 이그나이트가 사전 설치되어 있지 않은 상태이기 때문이다.  ~~참고로 pip 명령어는 package installer for python)의 약자이며, 아래 명령문 실행 시 이그나이트의 pre-release를 PyPI(python package index)로부터 설치하게 된다.~~ 

In [2]:
!pip install --pre pytorch-ignite

     |████████████████████████████████| 233 kB 5.4 MB/s 


결과에 따르면, 본 가이드라인 작성 시점의 최신 버전인 xxxx년 x월 xx일자 (예: 2021년 9월 10일자) 이그나이트 x.x.x (예: 0.5.0)가 설치 되었음을 알 수 있다.

####2.1.4.3. 딥러닝 코드

 필요한 환경이 적절히 준비되었으므로, 필요한 패키지를 아래 코드에서와 같이 로드한다.


In [3]:
from datetime import datetime
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim

from torch.cuda.amp import GradScaler, autocast
from torchvision import datasets, models
from torchvision.transforms import Compose, Normalize, Pad, RandomCrop, RandomHorizontalFlip, ToTensor

from torch.utils.tensorboard import SummaryWriter

import ignite
import ignite.distributed as idist
from ignite.contrib.engines import common
from ignite.contrib.handlers import PiecewiseLinear
from ignite.engine import Engine, Events
from ignite.handlers import Checkpoint, DiskSaver, global_step_from_engine
from ignite.metrics import Accuracy, Loss
from ignite.utils import manual_seed, setup_logger

 그리고 파이토치 사용 시와 동일하게 dataloader 처리를 진행한다. 이그나이트는 파이토치 기반의 wrapper이며, dataloader나 모델 architecturing 등은 기존 파이토치의 모델 학습 시의 pipeline 을 그대로 이용한다. 따라서 파이토치에 익숙하지 않을 경우 가이드라인 이해가 어려울 수 있음을 서문에서 언급한 바 있다.

 또한 이그나이트에서는 [auto_dataloader](https://pytorch.org/ignite/v0.4.5/generated/ignite.distributed.auto.auto_dataloader.html#auto-dataloader)함수 지원을 받아서, 다양한 환경(CPU, GPU, TPU)에서 분산 딥러닝 개발을 ~~그나마~~ 쉽게 진행할 수 있다.


In [4]:
train_transform = Compose(
    [
        Pad(4),
        RandomCrop(32, fill=128),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

test_transform = Compose([ToTensor(), Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),])


def get_train_test_datasets(path):
    if not os.path.exists(path):
        os.makedirs(path)
        download = True
    else:
        download = True if len(os.listdir(path)) < 1 else False

    train_dataset = datasets.CIFAR10(root=path, train=True, download=download, transform=train_transform)
    test_dataset = datasets.CIFAR10(root=path, train=False, download=False, transform=test_transform)

    return train_dataset, test_dataset

def get_data_loaders(config):
    if idist.get_local_rank() > 0:
        # Ensure that only local rank 0 download the dataset
        # Thus each node will download a copy of the dataset
        idist.barrier()

    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_dataset, test_dataset = get_train_test_datasets(config["data_path"])

    if idist.get_local_rank() == 0:
        # Ensure that only local rank 0 download the dataset
        idist.barrier()

    train_loader = idist.auto_dataloader(train_dataset, batch_size=config["batch_size"], num_workers=config["num_workers"], shuffle=True, drop_last=True,)
    val_loader = idist.auto_dataloader(test_dataset, batch_size=2 * config["batch_size"], num_workers=config["num_workers"], shuffle=False,)

    return train_loader, val_loader

 다음으로는 아래 코드에서와 같이 모델을 패키지로부터 로드하는 함수를 정의한다. 

In [5]:
def get_model(name):
    if name in models.__dict__:
        fn = models.__dict__[name]
    else:
        raise RuntimeError(f"Unknown model name {name}")

    return fn(num_classes=10)

로드 된 모델은 [auto_model](https://pytorch.org/ignite/v0.4.5/generated/ignite.distributed.auto.auto_model.html#auto-model) 함수의 지원을 받을 수 있도록 한다. 그리고 optimizer 관련 함수를 정의하고, 이 역시 [auto_optim](https://pytorch.org/ignite/v0.4.5/generated/ignite.distributed.auto.auto_optim.html#auto-optim)  함수의 지원을 받도록 한다..



In [6]:
def initialize(config):
    model = get_model(config["model"])
    # Adapt model for distributed settings if configured
    model = idist.auto_model(model)

    optimizer = optim.SGD(
        model.parameters(),
        lr=config["learning_rate"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"],
        nesterov=True,
    )
    optimizer = idist.auto_optim(optimizer)
    criterion = nn.CrossEntropyLoss().to(idist.device())

    le = config["num_iters_per_epoch"]
    milestones_values = [
        (0, 0.0),
        (le * config["num_warmup_epochs"], config["learning_rate"]),
        (le * config["num_epochs"], 0.0),
    ]
    lr_scheduler = PiecewiseLinear(optimizer, param_name="lr", milestones_values=milestones_values)

    return model, optimizer, criterion, lr_scheduler

아래 코드에서는 학습 중 정보를 얻을 수 있도록 log 출력 함수를 정의한다. 

In [7]:
def log_metrics(logger, epoch, elapsed, tag, metrics):
    metrics_output = "\n".join([f"\t{k}: {v}" for k, v in metrics.items()])
    logger.info(f"\nEpoch {epoch} - Evaluation time (seconds): {elapsed:.2f} - {tag} metrics:\n {metrics_output}")


def log_basic_info(logger, config):
    logger.info(f"Train {config['model']} on CIFAR10")
    logger.info(f"- PyTorch version: {torch.__version__}")
    logger.info(f"- Ignite version: {ignite.__version__}")
    if torch.cuda.is_available():
        # explicitly import cudnn as
        # torch.backends.cudnn can not be pickled with hvd spawning procs
        from torch.backends import cudnn

        logger.info(f"- GPU Device: {torch.cuda.get_device_name(idist.get_local_rank())}")
        logger.info(f"- CUDA version: {torch.version.cuda}")
        logger.info(f"- CUDNN version: {cudnn.version()}")

    logger.info("\n")
    logger.info("Configuration:")
    for key, value in config.items():
        logger.info(f"\t{key}: {value}")
    logger.info("\n")

    if idist.get_world_size() > 1:
        logger.info("\nDistributed setting:")
        logger.info(f"\tbackend: {idist.backend()}")
        logger.info(f"\tworld size: {idist.get_world_size()}")
        logger.info("\n")

학습을 수행하는 trainer와 evaluator 처리 함수를 정의한다.. 

In [8]:
def create_trainer(model, optimizer, criterion, lr_scheduler, train_sampler, config, logger):

    device = idist.device()

    # Setup Ignite trainer:
    # - let's define training step
    # - add other common handlers:
    #    - TerminateOnNan,
    #    - handler to setup learning rate scheduling,
    #    - ModelCheckpoint
    #    - RunningAverage` on `train_step` output
    #    - Two progress bars on epochs and optionally on iterations

    with_amp = config["with_amp"]
    scaler = GradScaler(enabled=with_amp)

    def train_step(engine, batch):

        x, y = batch[0], batch[1]

        if x.device != device:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

        model.train()

        with autocast(enabled=with_amp):
            y_pred = model(x)
            loss = criterion(y_pred, y)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        return {
            "batch loss": loss.item(),
        }

    trainer = Engine(train_step)
    trainer.logger = logger

    to_save = {"trainer": trainer, "model": model, "optimizer": optimizer, "lr_scheduler": lr_scheduler}
    metric_names = [
        "batch loss",
    ]

    common.setup_common_training_handlers(
        trainer=trainer,
        train_sampler=train_sampler,
        to_save=to_save,
        save_every_iters=config["checkpoint_every"],
        save_handler=get_save_handler(config),
        lr_scheduler=lr_scheduler,
        output_names=metric_names if config["log_every_iters"] > 0 else None,
        with_pbars=False,
        clear_cuda_cache=False,
    )

    resume_from = config["resume_from"]
    if resume_from is not None:
        checkpoint_fp = Path(resume_from)
        assert checkpoint_fp.exists(), f"Checkpoint '{checkpoint_fp.as_posix()}' is not found"
        logger.info(f"Resume from a checkpoint: {checkpoint_fp.as_posix()}")
        checkpoint = torch.load(checkpoint_fp.as_posix(), map_location="cpu")
        Checkpoint.load_objects(to_load=to_save, checkpoint=checkpoint)

    return trainer


def create_evaluator(model, metrics, config, tag="val"):
    with_amp = config["with_amp"]
    device = idist.device()

    @torch.no_grad()
    def evaluate_step(engine: Engine, batch):
        model.eval()
        x, y = batch[0], batch[1]
        if x.device != device:
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

        with autocast(enabled=with_amp):
            output = model(x)
        return output, y

    evaluator = Engine(evaluate_step)

    for name, metric in metrics.items():
        metric.attach(evaluator, name)

    if idist.get_rank() == 0:
        common.ProgressBar(desc=f"Evaluation ({tag})", persist=False).attach(evaluator)

    return evaluator

 아래 코드에서 모델 학습 상태를 저장할 수 있는 함수를 정의한다

In [9]:
def get_save_handler(config):
    return DiskSaver(config["output_path"], require_empty=False)

 그리고 모델 학습 관련 함수를 정의한다.

In [10]:
def training(local_rank, config, **kwargs):
    print(idist.get_rank(), ': run with config:', config, '- backend=', idist.backend())

    rank = idist.get_rank()
    # https://pytorch.org/docs/stable/notes/randomness.html
    manual_seed(config["seed"] + rank)
    # random.seed(random_seed) # torchvision.transform 사용 시 잊지 말 것
    # torch.cuda.manual_seed(random_seed)
    # torch.cuda.manual_seed_all(random_seed
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    device = idist.device()

    logger = setup_logger(name="CIFAR10-Training", distributed_rank=local_rank)

    log_basic_info(logger, config)

    output_path = config["output_path"]
    if rank == 0:
        if config["stop_iteration"] is None:
            now = datetime.now().strftime("%Y%m%d-%H%M%S")
        else:
            now = f"stop-on-{config['stop_iteration']}"

        folder_name = f"{config['model']}_backend-{idist.backend()}-{idist.get_world_size()}_{now}"
        output_path = Path(output_path) / folder_name
        if not output_path.exists():
            output_path.mkdir(parents=True)
        config["output_path"] = output_path.as_posix()
        logger.info(f"Output path: {config['output_path']}")

        if "cuda" in device.type:
            config["cuda device name"] = torch.cuda.get_device_name(local_rank)

    # Setup dataflow, model, optimizer, criterion
    train_loader, test_loader = get_data_loaders(config)

    config["num_iters_per_epoch"] = len(train_loader)
    model, optimizer, criterion, lr_scheduler = initialize(config)

    # Create trainer for current task
    trainer = create_trainer(model, optimizer, criterion, lr_scheduler, train_loader.sampler, config, logger)

    # Let's now setup evaluator engine to perform model's validation and compute metrics
    metrics = {
        "Accuracy": Accuracy(),
        "Loss": Loss(criterion),
    }

    # We define two evaluators as they wont have exactly similar roles:
    # - `evaluator` will save the best model based on validation score
    evaluator = create_evaluator(model, metrics=metrics, config=config)
    train_evaluator = create_evaluator(model, metrics=metrics, config=config)

    def run_validation(engine):
        epoch = trainer.state.epoch
        state = train_evaluator.run(train_loader)
        log_metrics(logger, epoch, state.times["COMPLETED"], "Train", state.metrics)
        state = evaluator.run(test_loader)
        log_metrics(logger, epoch, state.times["COMPLETED"], "Test", state.metrics)

    trainer.add_event_handler(Events.EPOCH_COMPLETED(every=config["validate_every"]) | Events.COMPLETED, run_validation)

    if rank == 0:
        # Setup TensorBoard logging on trainer and evaluators. Logged values are:
        #  - Training metrics, e.g. running average loss values
        #  - Learning rate
        #  - Evaluation train/test metrics
        evaluators = {"training": train_evaluator, "test": evaluator}
        tb_logger = common.setup_tb_logging(output_path, trainer, optimizer, evaluators=evaluators)

    # Store 2 best models by validation accuracy starting from num_epochs / 2:
    best_model_handler = Checkpoint(
        {"model": model},
        get_save_handler(config),
        filename_prefix="best",
        n_saved=2,
        global_step_transform=global_step_from_engine(trainer),
        score_name="test_accuracy",
        score_function=Checkpoint.get_default_score_fn("Accuracy"),
    )
    evaluator.add_event_handler(
        Events.COMPLETED(lambda *_: trainer.state.epoch > config["num_epochs"] // 2), best_model_handler
    )

    # In order to check training resuming we can stop training on a given iteration
    if config["stop_iteration"] is not None:

        @trainer.on(Events.ITERATION_STARTED(once=config["stop_iteration"]))
        def _():
            logger.info(f"Stop training on {trainer.state.iteration} iteration")
            trainer.terminate()

    try:
        trainer.run(train_loader, max_epochs=config["num_epochs"])
    except Exception as e:
        logger.exception("")
        raise e

    if rank == 0:
        tb_logger.close()

 이제 아래 코드에서와 같이, 상황에 따라 적절한 숫자만큼 프로세스를 생성하여 분산처리를 시행한다.


In [11]:
config = {
    "seed": 543,
    "data_path" : "./cifar10",
    "output_path" : "./output-cifar10/",
    "model" : "resnet18",
    "batch_size" : 512,
    "momentum" : 0.9,
    "weight_decay" : 1e-4,
    "num_workers" : 12,
    "num_epochs" : 24,
    "learning_rate" : 0.4,
    "num_warmup_epochs" : 4,
    "validate_every" : 3, 
    "checkpoint_every" : 1000,
    "backend" : None, 
    "resume_from" : None, 
    "log_every_iters" : 15,
    "nproc_per_node" : None, 
    "stop_iteration" : None, 
    "with_amp" : False
}

gpu_gtg = torch.cuda.is_available()

if not (tpu_gtg or gpu_gtg): # cpu
    config["backend"] = 'gloo'
    config["nproc_per_node"] = 8
elif gpu_gtg: # gpu
    config["backend"] = 'nccl'
    config["nproc_per_node"] = 1
elif tpu_gtg: # tpu
    config["backend"] = 'xla-tpu'
    config["nproc_per_node"] = 8
else: # error
    raise RuntimeError("Unknown environment: tpu_gtg {}, gpu_gtg {}".format(tpu_gtg, gpu_gtg))

if config["backend"] == "xla-tpu" and config["with_amp"]:
    raise RuntimeError("The value of with_amp should be False if backend is xla")

dist_configs = {'nproc_per_node': config["nproc_per_node"], "start_method": "fork"}  # or dist_configs = {...}

#print(config)
#print(dist_configs)
#print(config["backend"])

with idist.Parallel(backend=config["backend"], **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 07:29:07,370 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-09-11 07:29:07,372 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 07:29:07,375 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fa9bc123440>' in 8 processes


2 : run with config: {'seed': 543, 'data_path': './cifar10', 'output_path': './output-cifar10/', 'model': 'resnet18', 'batch_size': 512, 'momentum': 0.9, 'weight_decay': 0.0001, 'num_workers': 12, 'num_epochs': 24, 'learning_rate': 0.4, 'num_warmup_epochs': 4, 'validate_every': 3, 'checkpoint_every': 1000, 'backend': 'xla-tpu', 'resume_from': None, 'log_every_iters': 15, 'nproc_per_node': 8, 'stop_iteration': None, 'with_amp': False} - backend= xla-tpu
5 : run with config: {'seed': 543, 'data_path': './cifar10', 'output_path': './output-cifar10/', 'model': 'resnet18', 'batch_size': 512, 'momentum': 0.9, 'weight_decay': 0.0001, 'num_workers': 12, 'num_epochs': 24, 'learning_rate': 0.4, 'num_warmup_epochs': 4, 'validate_every': 3, 'checkpoint_every': 1000, 'backend': 'xla-tpu', 'resume_from': None, 'log_every_iters': 15, 'nproc_per_node': 8, 'stop_iteration': None, 'with_amp': False} - backend= xla-tpu
4 : run with config: {'seed': 543, 'data_path': './cifar10', 'output_path': './output-

2021-09-11 07:29:19,662 CIFAR10-Training INFO: Train resnet18 on CIFAR10
2021-09-11 07:29:19,682 CIFAR10-Training INFO: - PyTorch version: 1.9.0a0+gitc4c77e2
2021-09-11 07:29:19,683 CIFAR10-Training INFO: - Ignite version: 0.5.0.dev20210911
2021-09-11 07:29:19,687 CIFAR10-Training INFO: 

2021-09-11 07:29:19,691 CIFAR10-Training INFO: Configuration:
2021-09-11 07:29:19,695 CIFAR10-Training INFO: 	seed: 543
2021-09-11 07:29:19,700 CIFAR10-Training INFO: 	data_path: ./cifar10
2021-09-11 07:29:19,704 CIFAR10-Training INFO: 	output_path: ./output-cifar10/
2021-09-11 07:29:19,705 CIFAR10-Training INFO: 	model: resnet18
2021-09-11 07:29:19,715 CIFAR10-Training INFO: 	batch_size: 512
2021-09-11 07:29:19,718 CIFAR10-Training INFO: 	momentum: 0.9
2021-09-11 07:29:19,723 CIFAR10-Training INFO: 	weight_decay: 0.0001
2021-09-11 07:29:19,728 CIFAR10-Training INFO: 	num_workers: 12
2021-09-11 07:29:19,730 CIFAR10-Training INFO: 	num_epochs: 24
2021-09-11 07:29:19,738 CIFAR10-Training INFO: 	learning

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10


...

2021-09-11 07:46:38,592 CIFAR10-Training INFO: 
Epoch 24 - Evaluation time (seconds): 4.01 - Test metrics:
 	Accuracy: 0.8112
	Loss: 0.54418720703125
2021-09-11 07:46:38,621 CIFAR10-Training INFO: Epoch[24] Complete. Time taken: 00:01:00


Evaluation (val)[1/97]   1%|1          [00:00<?]

2021-09-11 07:47:03,045 CIFAR10-Training INFO: 
Epoch 24 - Evaluation time (seconds): 24.41 - Train metrics:
 	Accuracy: 0.8413740335051546
	Loss: 0.45592994296673645


Evaluation (val)[1/10]  10%|#          [00:00<?]

2021-09-11 07:47:07,003 CIFAR10-Training INFO: 
Epoch 24 - Evaluation time (seconds): 3.93 - Test metrics:
 	Accuracy: 0.8112
	Loss: 0.54418720703125
2021-09-11 07:47:07,019 CIFAR10-Training INFO: Engine run complete. Time taken: 00:17:27
2021-09-11 07:47:07,152 ignite.distributed.launcher.Parallel INFO: End of run


 CPU단인 경우, 결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 glool 백엔드로 초기화 되었음을 표시한다.

>*2021-07-05 06:52:37,194 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'*  
...


 GPU단인 경우, 결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 nccl 백엔드로 초기화 되었음을 표시한다.

>*2021-07-05 04:44:07,708 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'nccl'*  
...


 TPU단인 경우, 결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 xla-tpu 백엔드로 초기화 되었음을 표시한다.

>*2021-07-05 07:00:09,365 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'*  
...



 상기 ‘’’1.4 분산 딥러닝 기본 지식’’’ 항목에서 언급된 바와 같이 분산처리를 위해서는 컴퓨팅 코어 간 통신이 필요하며, 사용된 컴퓨팅 코어 종류에 따라 CPU는 gloo, GPU는 nccl, TPU는  xla를 백엔드로 지정한 것이다. 

 이와 관련해서는 DISTRIBUTED COMMUNICATION PACKAGE - TORCH.DISTRIBUTED 페이지의 [rule of thumb](https://pytorch.org/docs/stable/distributed.html) 항목을 참조한다. ~~분산처리의 역사만큼이나 다양한 분산처리 방식이 존재한다. TCP 등을 이용해 직접 프로세스간 통신을 처리하는 방법도 있지만, 하이레벨 관점에서 CPU는 gloo, GPU는 nccl, TPU는 xla 백엔드를 사용해야 한다고 생각하는 것이 정신건강에 좋다.~~

 CPU단인 경우 두번 째 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 8개의 처리 작업(nproc_per_node: 8)이 진행될 것임을 알려준다. 그리고 2.1.1.5 결과 해석 항목에서 설명된 바와 같이, CPU와 같은 멀티코어 에서는 동시에 작업을 하는 것처럼 보이는 것([concurrent computing](https://en.wikipedia.org/wiki/Concurrent_computing))이 가능하므로, 노드 당 처리 작업의 수를 더욱 크게 늘려도 상관 없다. 하지만 VM에 할당된 코어가 2개에 불과하므로 분산처리 효과를 얻기 어려운 일이다. 

>...  
*2021-07-05 06:52:37,195 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
	>>*nproc_per_node: 8*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  
...



GPU단인 경우 두번 째 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 1개의 처리 작업(nproc_per_node: 1)이 진행될 것임을 알려준다.  이는 2.1.2.5 결과 해석 항목에서 설명된 바와 같이, colab에서 할당 받은 VM에 포함된 GPU가 1장이고, 멀티코어인 CPU에서와 달리 매니코어인 GPU에서는 1장 당 처리되는 프로세스를 1개로 지정되기 때문이다.

>...  
*2021-07-05 04:44:07,709 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
	>>*nproc_per_node: 1*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  
...  



TPU단인 경우 두번 째 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 8개의 처리 작업(nproc_per_node: 8)이 진행될 것임을 알려준다. 2.1.3.5 결과 해석 항목에서 설명된 바와 같이 colab에서 할당되는 TPU 1장에는 8개의 코어가 탑재되어 있어으로, 프로세스 8개를 이용할 수 있다. 

>...  
>*2021-07-05 07:00:09,370 이그나이트.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
	>>*nproc_per_node: 8*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  
... 



 이제 epoch당 처리 속도를 살펴보면 다음과 같다.

 먼저 CPU단인 경우에는, 약 11분 20초에서 11분 44초가 소요되었다.

>...  
*2021-07-05 07:04:23,800 CIFAR10-Training INFO: Epoch[1] Complete. Time taken: 00:11:20*  
*2021-07-05 07:16:07,949 CIFAR10-Training INFO: Epoch[2] Complete. Time taken: 00:11:44*  
*2021-07-05 07:29:31,288 CIFAR10-Training INFO:*   
*Epoch 3 - Evaluation time (seconds): 115.12 - Train metrics:*
 	>>*Accuracy: 0.5924210695876289*  
	*Loss: 1.1792468041488804*  
...



 그리고 GPU단인 경우에는, 약 18초가 소요되었다.

>...  
*2021-07-05 04:44:31,622 CIFAR10-Training INFO: Epoch[1] Complete. Time taken: 00:00:18*  
*2021-07-05 04:44:49,820 CIFAR10-Training INFO: Epoch[2] Complete. Time taken: 00:00:18*  
*2021-07-05 04:45:22,482 CIFAR10-Training INFO:*  
*Epoch 3 - Evaluation time (seconds): 13.91 - Train metrics:*  
 	>>*Accuracy: 0.5915351159793815*  
	*Loss: 1.1530251257198374*  
...  



 TPU단인 경우에는, 약 30초에서 32초가 소요되었다.

>...  
*2021-07-05 07:01:34,136 CIFAR10-Training INFO: Epoch[1] Complete. Time taken: 00:00:30*  
*2021-07-05 07:02:05,283 CIFAR10-Training INFO: Epoch[2] Complete. Time taken: 00:00:32*  
*2021-07-05 07:03:00,600 CIFAR10-Training INFO:*  
*Epoch 3 - Evaluation time (seconds): 24.52 - Train metrics:*  
 	>>*Accuracy: 0.4245932667525773*  
	*Loss: 1.612633144732603*  
...




전체 처리 시간은 다음과 같다.

 먼저 CPU단인 경우에는, 아직도 계산중이다. -_-*

>...  
...


 GPU단은 다음과 같이 약 10분 25초가 소요되었다.

>...  
*2021-07-05 04:54:38,550 CIFAR10-Training INFO: Engine run complete. Time taken: 00:10:25*  
*2021-07-05 04:54:39,024 ignite.distributed.launcher.Parallel INFO: End of run*

 TPU단은 다음과 같이 약 6분 44초가 소요되었다

>...  
*2021-07-05 07:18:08,730 CIFAR10-Training INFO: Engine run complete. Time taken: 00:06:44*  
*2021-07-05 07:18:08,843 ignite.distributed.launcher.Parallel INFO: End of run*  


이를 도표로 정리하면 다음과 같다.

<div align="center">
<img width=720 src="https://i.imgur.com/FS5FQdR.png"/>
</div>



 해당 시험에서는 GPU와 TPU 코어에 여러 가지 종류의 레이턴시(latency)가 발생했음을 짐작할 수 있다. 


Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  